In [1]:
# python -m venv policyengine_venv
# source policyengine_venv/bin/activate
# make sure to select kernel to be in the above environment
# Install ipykernel if not already installed
!pip install ipykernel

# Register the environment as a kernel
!python -m ipykernel install --user --name=policyengine_venv --display-name="Python (PolicyEngine)"
# import sys
# print(sys.executable)  # Should show a path inside your policyengine_venv directory

Installed kernelspec policyengine_venv in /Users/helenauta.bolt/Library/Jupyter/kernels/policyengine_venv


In [ ]:
#Create environment variable for huggingface token
#Install policy engine
!pip install policyengine-uk

In [4]:
import os

#os.environ["HUGGING_FACE_TOKEN"] = 

In [ ]:
# The `Simulation` class is the most important class in PolicyEngine- it runs the actual simulation.
import sys
print(f"Current Python: {sys.executable}")
import policyengine_uk
print(f"PolicyEngine UK version: {policyengine_uk.__file__}")
from policyengine_uk import Simulation

situation = {
    "people": {
        "person": {
            "age": {2025: 30},
            "employment_income": {2025: 30_000},
        },
    },
    "benunits": {
        "benunit": {
            "members": ["person"],
        },
    },
    "households": {
        "household": {
            "members": ["person"],
        }
    },
}

simulation = Simulation(situation=situation)

simulation.calculate("income_tax", 2025)

Current Python: /Users/helenauta.bolt/Documents/GitHub/policyengine-uk/policyengine_venv/bin/python


/Users/helenauta.bolt/Documents/GitHub/policyengine-uk/policyengine_venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


PolicyEngine UK version: /Users/helenauta.bolt/Documents/GitHub/policyengine-uk/policyengine_uk/__init__.py


array([3486.], dtype=float32)

In [ ]:
from policyengine_core.model_api import *


increase_basic_rate = {"gov.hmrc.income_tax.rates.uk[0].rate": 0.25}


baseline = Simulation(situation=situation)
reformed = Simulation(situation=situation, reform=increase_basic_rate)

baseline_income_tax = baseline.calculate("income_tax", 2025)[0]
reformed_income_tax = reformed.calculate("income_tax", 2025)[0]
print(f"Pre reform, this individual payed an income tax of:{baseline_income_tax}")
print(
    f"Raising the basic rate to 25% would increase this person's income tax by £{reformed_income_tax - baseline_income_tax:.2f}"
)

In [ ]:
from policyengine_uk import Microsimulation
LOCAL_enhanced_frs_2022_23="hf://policyengine/policyengine-uk-data/enhanced_frs_2022_23.h5"
#sim = Microsimulation(dataset="hf://policyengine/policyengine-uk-data/enhanced_frs_2022_23.h5")
sim = Microsimulation(dataset=LOCAL_enhanced_frs_2022_23)
# The hf:// points to the private data-
# hf:// <- go get the data from huggingface
# policyengine/policyengine-uk-data <- the repo
# enhanced_frs_2022_23.h5 <- the file in the repo

sim.calculate("universal_credit", 2025).sum() / 1e9

In [3]:
from policyengine_uk import Microsimulation
LOCAL_enhanced_frs_2022_23="hf://policyengine/policyengine-uk-data/enhanced_frs_2022_23.h5"
#sim = Microsimulation(dataset="hf://policyengine/policyengine-uk-data/enhanced_frs_2022_23.h5")
sim = Microsimulation(dataset=LOCAL_enhanced_frs_2022_23)
# The hf:// points to the private data-
# hf:// <- go get the data from huggingface
# policyengine/policyengine-uk-data <- the repo
# enhanced_frs_2022_23.h5 <- the file in the repo

sim.calculate("universal_credit", 2025).sum() / 1e9

75.76009336812831

In [ ]:
sim = Microsimulation(dataset="/Users/helenauta.bolt/Documents/GitHub/policyengine-uk-data/policyengine_uk_data/storage/enhanced_frs_2022_23.h5")

df = sim.calculate_dataframe(
    [
        "household_id", # If the first variable is household level, the dataframe will project everything to households. Same for people.
        "income_tax",
        "region",
    ],
    period=2025
)

df.groupby("region").income_tax.sum().sort_values(ascending=False)/1e9 # Weights automatically applied

In [ ]:
from policyengine_uk.model_api import *
enhanced_frs="/Users/helenauta.bolt/Documents/GitHub/policyengine-uk-data/policyengine_uk_data/storage/enhanced_frs_2022_23.h5"


def change_tax_parameters(parameters):
    parameters.gov.hmrc.income_tax.rates.uk.brackets[0].rate.update(
        period=periods.period("year:2019:10"), value=0.23
    )
    return parameters


class reform(Reform):
    def apply(self):
        self.modify_parameters(change_tax_parameters)


baseline = Microsimulation(dataset=enhanced_frs)
reformed = Microsimulation(dataset=enhanced_frs, reform=reform)
revenue = reformed.calculate("gov_balance", 2025).sum() - baseline.calc("gov_balance", 2025).sum()
f"Revenue: £{round(revenue / 1e+9, 1)}bn"

In [ ]:
from policyengine_uk import Microsimulation
from policyengine_core.reforms import Reform
from policyengine_core.periods import instant
import pandas as pd


reform = Reform.from_dict({
  "gov.dwp.universal_credit.elements.child.limit.child_count": {
    "2023-01-01.2028-12-31": 99
  }
}, country_id="uk")
 
baseline = Microsimulation()
reformed = Microsimulation(reform=reform)


In [ ]:
baseline_income = baseline.calculate("household_net_income", period=2025)
reformed_income = reformed.calculate("household_net_income", period=2025)
difference_income = reformed_income - baseline_income
HOUSEHOLD_VARIABLES = ["person_id", "household_id", "age", "household_net_income", "household_income_decile", "in_poverty", "household_tax", "household_benefits"]
baseline_person_df = baseline.calculate_dataframe(HOUSEHOLD_VARIABLES, 2025).astype(float)
reformed_person_df = reformed.calculate_dataframe(HOUSEHOLD_VARIABLES, 2025).astype(float)
difference_person_df = reformed_person_df - baseline_person_df
total_net_income_baseline = baseline.calculate("household_net_income", 2025)
df=pd.DataFrame({"value":total_net_income_baseline.values, "weights":total_net_income_baseline.weights})
df.to_csv("baseline_income.csv")
total_net_income_baseline = baseline.calculate("household_net_income", 2025).sum()
total_net_income_reformed = reformed.calculate("household_net_income", 2025).sum()

net_cost = total_net_income_reformed - total_net_income_baseline

print(f"This reform would cost £{net_cost / 1e9:.1f}bn")
 

In [ ]:
### By oldest person in HH
# Step 1: Get oldest person in each household
baseline_person_df = baseline.calculate_dataframe(HOUSEHOLD_VARIABLES, 2025).astype(float)
reformed_person_df = reformed.calculate_dataframe(HOUSEHOLD_VARIABLES, 2025).astype(float)
baseline_idx = baseline_person_df.groupby("household_id")["age"].idxmax()
reformed_idx = reformed_person_df.groupby("household_id")["age"].idxmax()
baseline_df = baseline_person_df.loc[baseline_idx]
reformed_df = reformed_person_df.loc[reformed_idx]
# Step 2: Merge the two datasets
df_all = pd.merge(
    baseline_df,
    reformed_df,
    on="household_id",
    suffixes=("_baseline", "_reformed")
)
#`df_all["weights"]=total_net_income_baseline.weights

# Step 3: Create age bands from the oldest person's age
bins = [0, 24, 34, 44, 54, 64, 74, 200]
labels = ["<25", "25-34", "35-44", "45-54", "55-64", "65-74", "75+"]
df_all["age_group"] = pd.cut(df_all["age_baseline"], bins=bins, labels=labels, right=False)
# Step 4: Calculate net cost per household
df_all["net_cost"] = (df_all["household_net_income_reformed"] - df_all["household_net_income_baseline"])* df_all["weights"]
# Step 5: Group by age group and sum net cost
net_cost_by_age_group = df_all.groupby("age_group")["net_cost"].sum()
# Show the result
print(net_cost_by_age_group/ 1e9)
print(net_cost_by_age_group.sum()/ 1e9)
 